In [4]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from konlpy.tag import Okt         ; t = Okt()     # 구 트위터
from konlpy.tag import *

import nltk
import pickle

In [5]:
df_with = pd.read_csv('./with_name/all_headlines.tsv', sep="\t")

In [6]:
df_with['뉴스제목'] = df_with['뉴스제목'].str.replace('\s+', ' ', regex=True)
df_with['뉴스제목'] = df_with['뉴스제목'].map(lambda x: re.sub('[0-9,.!·,…*>%@#&$<_?.:\“\/\”\’\'\"\^\\‘\|\`\(\)\[\]\{\}]', '', x))
df_with['뉴스제목'] = df_with['뉴스제목'].map(lambda x: re.sub('\u200c', '', x))
df_with['뉴스제목'] = df_with['뉴스제목'].map(lambda x: re.sub('[-–ㆍ]', ' ', x))

stop_words = ['\u200c','\n ','\n',"'",'…',',','[',']','(',')','"','주','에','·','장','-','적',\
              '도','‘','`','가','’','의','이','★','은','“','대','”','한','B','로',\
              '?','선','A','는','!','"…','상','들','제','…"','일','서','명',"'…",'기',\
              '···','소','등','자','전','률','미','...','세','시','안','폭',"…'",'만','억',\
              '눈','더','량','고','인','성','다','감','을','지','수','것','째',\
              '기','···','중','계','왜','총','내','과','젠','또','연','엔','차','할',\
              '새','사','때','..','임','속','’…','G','나','개','원',\
             '달','→','권','?…','간','배','K','저','와','하','/','조','두','분','형',\
             '황','공','&','보','문','익','X','억원',']"','치','산','를','오','해','S','그','된','준','▶',\
             '건','재','반','라','년','초','분','월','신','p','급','줄','경','구','진','올','발','vs','강',\
             '국','난','판','면','"(','`…','살','아','번','텍','팜','Q','메','점','월',\
             'D','비','됐다','채',"]'",'보니','손','확','종','동','팔','타','~','땐','말','요',\
             "',",'스','…`','단','길','회','호','용','듯','최']

df_with['뉴스제목'] = [each_word for each_word in df_with['뉴스제목'] if each_word not in stop_words]

In [8]:
df_with['okt'] = df_with.apply(lambda row: t.morphs(row['뉴스제목']), axis=1)
df_with

,뉴스제목,주가변동,okt
0,LG생활건강 숨° 로시크숨마 황제의 빛 에디션 선봬,1,"[LG, 생활, 건강, 숨, °, 로, 시크, 숨마, 황제, 의, 빛, 에디, 션,..."
1,IRSK이노베이션 내년 시설투자 올해보다 낮은 수준,1,"[IRSK, 이노베이션, 내년, 시설, 투자, 올해, 보다, 낮은, 수준]"
2,수출입銀 CJ대한통운 해외채권 발행 공동 보증,0,"[수출입, 銀, CJ, 대, 한, 통운, 해외, 채권, 발행, 공동, 보증]"
3,브랜드 아파트 만원대에 마련할 마지막 기회 사천 KCC스위첸,1,"[브랜드, 아파트, 만원, 대, 에, 마련, 할, 마지막, 기회, 사천, KCC, ..."
4,LG전자 뉴욕 한복판에서 영화 상영한 사연,0,"[LG, 전자, 뉴욕, 한, 복판, 에서, 영화, 상영, 한, 사연]"
...,...,...,...
611468,한국항공우주 인니에 억원 규모 KT B 훈련기 공급 계약,1,"[한국, 항공우주, 인니, 에, 억원, 규모, KT, B, 훈련기, 공급, 계약]"
611469,하이트진로 세 박태영 사장 승진,1,"[하이트진로, 세, 박태영, 사장, 승진]"
611470,특징주 형제의 난 재점화 가능성에롯데지주 강세,0,"[특징, 주, 형제, 의, 난, 재, 점화, 가능성, 에, 롯데, 지주, 강세]"
611471,제일기획 비상교육과 청소년 대상 마음 교과서 개발,1,"[제일기획, 비상교육, 과, 청소년, 대상, 마음, 교과서, 개발]"


https://wikidocs.net/24949

In [11]:
from gensim import corpora
dictionary = corpora.Dictionary(df_with['okt'])
corpus = [dictionary.doc2bow(text) for text in df_with['okt']]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]


In [12]:
print(dictionary[66])

동반성


In [13]:
len(dictionary)

59800

In [14]:
import gensim
NUM_TOPICS = 2 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.012*"LG" + 0.012*"에" + 0.008*"전자" + 0.008*"현대"')
(1, '0.019*"투자" + 0.016*"증권" + 0.014*"에" + 0.012*"억"')


In [15]:
print(ldamodel.print_topics())

[(0, '0.012*"LG" + 0.012*"에" + 0.008*"전자" + 0.008*"현대" + 0.008*"삼성" + 0.007*"SK" + 0.007*"출시" + 0.006*"CJ" + 0.006*"건설" + 0.006*"한"'), (1, '0.019*"투자" + 0.016*"증권" + 0.014*"에" + 0.012*"억" + 0.012*"원" + 0.012*"주" + 0.011*"뉴스" + 0.010*"가" + 0.010*"분기" + 0.010*"영업"')]


In [18]:
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\MG\Python\Python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [19]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.9636511), (1, 0.03634888)]
1 번째 문서의 topic 비율은 [(0, 0.3047449), (1, 0.6952551)]
2 번째 문서의 topic 비율은 [(0, 0.57048565), (1, 0.42951438)]
3 번째 문서의 topic 비율은 [(0, 0.95033354), (1, 0.049666516)]
4 번째 문서의 topic 비율은 [(0, 0.942805), (1, 0.057195038)]


In [20]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [21]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

In [ ]:
with open('./with_name/LDA_topictable.pkl', 'wb') as f:
    pickle.dump(topictable, f)